In [1]:
import pandas as pd
from selenium import webdriver
import time
from tqdm import tqdm

In [2]:
train = pd.read_csv("./data/train.csv")

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(chrome_options=options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [4]:
def trans(t, driver):
    new_text = []
    lang = ["ko", 'ja&hn=0']
    for l in lang:
        backtrans = ""
        try:
            driver.get('https://papago.naver.com/?sk=en&tk='+l+'&st='+t)
            time.sleep(2.5)
            backtrans = driver.find_element_by_xpath('//*[@id="txtTarget"]').text
        except:
            pass
        if backtrans != "":
            return_sen = ""
            try:
                driver.get('https://papago.naver.com/?sk='+l+'&tk=en&st='+backtrans)
                time.sleep(2.5)
                return_sen = driver.find_element_by_xpath('//*[@id="txtTarget"]').text
            except:
                pass
            if return_sen != "":
                new_text.append(return_sen)
    return list(set(new_text))


In [7]:
def backtrans_total(driver):
    train = pd.read_csv("./data/train.csv")
    val = train["label"].values.tolist()
    val = list(set(val))

    aug_label = []
    for v in val:
        if len(train[train["label"] == v]) < 1000:
            aug_label.append(v)

    aug_data = pd.DataFrame()
    for idx in tqdm(range(len(train))):
        original = train.iloc[idx]
        if original["label"] in aug_label:
            new_text = trans(original["문장"], driver)
            for n in new_text:
                if original["문장"] != n:
                    add = pd.DataFrame([{
                        "ID" : original["ID"],
                        "문장": n,
                        "유형": original["유형"],
                        "극성": original["극성"],
                        "시제": original["시제"],
                        "확실성": original["확실성"],
                        "label": original["label"]
                    }])
                    aug_data = aug_data.append(add)

    train = train.append(aug_data)
    train.to_csv("./data/train_totalaug.csv")


In [8]:
backtrans_total(driver)

100%|██████████| 16541/16541 [10:14:16<00:00,  2.23s/it] 


In [9]:
def backtranslation(driver):
    train = pd.read_csv("./data/train.csv")
    train.drop(["극성", "시제", "확실성", "label"], axis=1)

    aug_label = ["추론형", "대화형", "예측형"]
    aug_data = pd.DataFrame()
    for a in tqdm(aug_label):
        aug_data = aug_data.append(train[train["유형"] == a])
    
    auged_data = pd.DataFrame()
    for idx in tqdm(range(len(aug_data))):
        original = aug_data.iloc[idx]
        new_text = trans(original["문장"], driver)
        for n in new_text:
            if original["문장"] != n:
                add = pd.DataFrame([{
                    "ID" : original["ID"],
                    "문장" : n,
                    "유형" : original["유형"],
                }])
                auged_data = auged_data.append(add)
    train = train.append(auged_data)
    train.to_csv("./data/train_backtrans_0.csv")


In [10]:
backtranslation(driver)

100%|██████████| 2983/2983 [8:39:40<00:00, 10.45s/it]  


In [11]:
# 극성
def backtranslation_1(driver):
    train = pd.read_csv("./data/train.csv")
    train.drop(["유형", "시제", "확실성", "label"], axis=1)

    aug_label = ["부정", "미정"]
    aug_data = pd.DataFrame()
    for a in tqdm(aug_label):
        aug_data = aug_data.append(train[train["극성"] == a])
    
    auged_data = pd.DataFrame()
    for idx in tqdm(range(len(aug_data))):
        original = aug_data.iloc[idx]
        new_text = trans(original["문장"], driver)
        for n in new_text:
            if original["문장"] != n:
                add = pd.DataFrame([{
                    "ID" : original["ID"],
                    "문장" : n,
                    "극성" : original["극성"],
                }])
                auged_data = auged_data.append(add)
    train = train.append(auged_data)
    train.to_csv("./data/train_backtrans_1.csv")


In [12]:
backtranslation_1(driver)

100%|██████████| 748/748 [2:10:18<00:00, 10.45s/it]  


In [13]:
# 시제
def backtranslation_2(driver):
    train = pd.read_csv("./data/train.csv")
    train.drop(["유형", "극성", "확실성", "label"], axis=1)

    aug_label = ["미래"]
    aug_data = pd.DataFrame()
    for a in tqdm(aug_label):
        aug_data = aug_data.append(train[train["시제"] == a])
    
    auged_data = pd.DataFrame()
    for idx in tqdm(range(len(aug_data))):
        original = aug_data.iloc[idx]
        new_text = trans(original["문장"], driver)
        for n in new_text:
            if original["문장"] != n:
                add = pd.DataFrame([{
                    "ID" : original["ID"],
                    "문장" : n,
                    "시제" : original["시제"],
                }])
                auged_data = auged_data.append(add)
    train = train.append(auged_data)
    train.to_csv("./data/train_backtrans_2.csv")

In [14]:
backtranslation_2(driver)

100%|██████████| 1643/1643 [4:45:49<00:00, 10.44s/it]  


In [15]:
# 확실성
def backtranslation_3(driver):
    train = pd.read_csv("./data/train.csv")
    train.drop(["유형", "극성", "시제", "label"], axis=1)

    aug_label = ["불확실"]
    aug_data = pd.DataFrame()
    for a in tqdm(aug_label):
        aug_data = aug_data.append(train[train["확실성"] == a])
    
    auged_data = pd.DataFrame()
    for idx in tqdm(range(len(aug_data))):
        original = aug_data.iloc[idx]
        new_text = trans(original["문장"], driver)
        for n in new_text:
            if original["문장"] != n:
                add = pd.DataFrame([{
                    "ID" : original["ID"],
                    "문장" : n,
                    "확실성" : original["확실성"],
                }])
                auged_data = auged_data.append(add)
    train = train.append(auged_data)
    train.to_csv("./data/train_backtrans_3.csv")

In [ ]:
backtranslation_3(driver)